### モデルを作成する

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch.utils.data as data
import torch.optim as optim

import cv2
import numpy as np
import time
import ast
import random
import matplotlib.pyplot as plt
import pandas as pd

ModuleNotFoundError: No module named 'torch'

In [2]:
from utils.ssd_model import SSD
from utils.ssd_model import MultiBoxLoss

In [4]:
input_size = 300
color_mean = (0, 0)
voc_classes = ['ring']

In [5]:
ssd_cfg = {
    'num_classes': 2,  # 背景クラスを含めた合計クラス数
    'input_size': input_size,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 4, 4, 4, 4, 4],  # 出力するDBoxのアスペクト比の種類
#      'bbox_aspect_num': [4, 6, 6, 6, 4, 4]
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ   
#     'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2], [2], [2], [2], [2]],
#     'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

In [23]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [24]:
net = SSD(phase='train', cfg=ssd_cfg)

In [27]:
net.to(device)
summary(net, (2, 300, 300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 300, 300]             152
              ReLU-2          [-1, 8, 300, 300]               0
         MaxPool2d-3          [-1, 8, 100, 100]               0
            Conv2d-4         [-1, 16, 100, 100]           1,168
              ReLU-5         [-1, 16, 100, 100]               0
         MaxPool2d-6           [-1, 16, 50, 50]               0
            Conv2d-7           [-1, 32, 46, 46]          12,832
              ReLU-8           [-1, 32, 46, 46]               0
            Conv2d-9           [-1, 32, 42, 42]          25,632
             ReLU-10           [-1, 32, 42, 42]               0
           Conv2d-11           [-1, 32, 38, 38]          25,632
             ReLU-12           [-1, 32, 38, 38]               0
           L2Norm-13           [-1, 32, 38, 38]              32
        MaxPool2d-14           [-1, 32,